|       | Description |
| ----------- | ----------- |
| Author      | Blaine Hill (MSCS student) |
| Date   | 11/25/2023        |
|Email|blaineh2@illinois.edu|
|Idea| Use Selenium to get the news contents from WSJ |

In [ ]:
from selenium import webdriver # allow launching browser
from selenium.webdriver.common.by import By # allow search with parameters
from selenium.webdriver.support.ui import WebDriverWait # allow waiting for page to load
from selenium.webdriver.support import expected_conditions as EC # determine whether the web page has loaded
from selenium.common.exceptions import TimeoutException # handling timeout situation
import time
import os
from random import uniform, random, choice, shuffle
import pandas as pd
from tqdm import tqdm
import pickle

In [ ]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)

In [ ]:
def create_webdriver():
    return webdriver.chrome.webdriver.WebDriver()

In [ ]:
%load_ext dotenv
%dotenv

email = %env EMAIL
PW = %env PASSWORD 

In [ ]:
browser = create_webdriver()
link = "https://sso.accounts.dowjones.com/login-page?client_id=5hssEAdMy0mJTICnJNvC9TXEw3Va7jfO&redirect_uri=https%3A%2F%2Fwww.wsj.com%2Fclient%2Fauth&response_type=code&scope=openid%20idp_id%20roles%20tags%20email%20given_name%20family_name%20uuid%20djid%20djUsername%20djStatus%20trackid%20prts%20updated_at%20created_at%20offline_access&ui_locales=en-us-x-wsj-223-2&nonce=52c47041-ffac-4ebd-a213-e777ff5b684e&state=Br3BYl9iSECXktJn.c8ksuGzlG5CBOw5mEnxqxAx_vb3PwwuwocDA8Tqt_CU&resource=https%253A%252F%252Fwww.wsj.com%252F&protocol=oauth2&client=5hssEAdMy0mJTICnJNvC9TXEw3Va7jfO#!/signin"
browser.get(link)

In [ ]:
email_field = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.ID, "username")))
email_field.send_keys(email)

In [ ]:
l = browser.find_element(By.XPATH, '//*[@id="basic-login"]/div[1]/form/div[2]/div[6]/div[1]/button[1]')
browser.execute_script("arguments[0].click();", l);

In [ ]:
pw_field =  WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="password-login-password"]')))
pw_field.send_keys(PW)

In [ ]:
login_button = browser.find_element(By.XPATH, '//*[@id="password-login"]/div/form/div/div[5]/div[1]/button')
browser.execute_script("arguments[0].click();", login_button);

In [ ]:
keyword = "google"
link2 = f"https://www.wsj.com/search?query={keyword}&isToggleOn=true&operator=OR&sort=relevance&duration=1y&startDate=2022%2F03%2F29&endDate=2023%2F10%2F16&source=wsjie%2Cblog%2Cwsjvideo%2Cinteractivemedia%2Cwsjsitesrch%2Cwsjpro%2Cwsjaudio%2Clivecoverage"
browser.get(link2)

In [ ]:
dir_path = f"../data/Links for Finetuning Language Models/"
os.makedirs(dir_path, exist_ok=True)
file_name = f"wsj_{keyword}_links.txt"
file_path = os.path.join(dir_path, file_name)


In [ ]:
links = []
for i in range(500):
    elems = browser.find_elements(By.XPATH,"//a[@href]")
    tmp_set = {elem.get_attribute("href") for elem in elems if "wsj.com/articles/" in elem.get_attribute("href")}
    links += list(tmp_set)
    with open(file_path, 'a') as f:
        # Write each link to the file, followed by a newline character
        for link in tmp_set:
            f.write(link + '\n')
    browser.get(link2 + f"&page={i+2}")
    print("reaching page: ", i + 1, 'len(links):', len(links))

    time.sleep(choice([(random() + 0.5) * 20, (random() + 0.5) * 30]))